<p style="font-family: Arial; font-size:2.75em;color:blue; font-style:bold"><br>
Projet 6 :<br><br>
Catégorisez automatiquement des questions<br><br>
</p><br>

* **Data Source / Data Source :** 
    - Data :
        - Outils d'extraction de data : https://data.stackexchange.com/stackoverflow
        - Script : https://github.com/EricJacquesPro/TextCategorization/blob/master/SQL/ExtractionDonnees.sql
    - Source Python : https://github.com/EricJacquesPro/TextCategorization
* **Description / Description : ** Projet 6 de la formation Data Scientist Proposée par OpenClassroom
    - Analyse LDA
    - Entrainement LDA
* **Auteur / Author : ** Eric JACQUES 
* **Date : ** 18/07/2020 

In [1]:
import sys
import os
import numpy as np
import pandas as pd
sys.path.insert(0, './Python/')

In [2]:
'''
python class for the data engineering (reading, cleaning, training...)
localised in ./Python
'''
from tagText import TagText 

C:\Users\naru_\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [3]:
tagText = TagText()
print(tagText.urlDirectory)#folder for cvs file generated by SQL query (cf. I) 
print(tagText.fileName)#name of the cvs file generated by SQL query (cf. I)

tagText.nombre_post_entree = 50000
tagText.precision = 50000

data_question = tagText.read_source()
data_question.head()


Data/
QuestionVsTags.csv


,Id,body,title,tags,SelectedTags,Name
0,11227809,<p>Here is a piece of C++ code that shows some...,Why is processing a sorted array faster than p...,<java><c++><performance><optimization><branch-...,"java,c++,performance,optimization,",Question
1,927358,<p>I accidentally committed the wrong files to...,How do I undo the most recent local commits in...,<git><version-control><git-commit><undo><pre-c...,"git,version-control,git-commit,",Question
2,2003505,<p>I want to delete a branch both locally and ...,How do I delete a Git branch locally and remot...,<git><version-control><git-branch><git-push><g...,"git,version-control,git-branch,git-push,",Question
3,292357,<p>What are the differences between <code>git ...,What is the difference between 'git pull' and ...,<git><version-control><git-pull><git-fetch>,"git,version-control,",Question
4,477816,"<p>I've been messing around with <a href=""http...",What is the correct JSON content type?,<json><http-headers><content-type>,"json,http-headers,content-type,",Question


In [4]:
data_preprocessed = data_question['body'].apply(tagText.preprocessing)
print(data_preprocessed[1:9])

1    accidentally committed wrong files git pushed ...
2    want delete branch locally remotely failed att...
3                       differences git pull git fetch
4    ive messing around json time pushing text hurt...
5    use yield keyword python do example im trying ...
6    mistakenly added files git using command git a...
7    reading hidden features dark corners cstl comp...
8    want rename remote branch described rename mas...
Name: body, dtype: object


<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
III.2 - Train/Test</p>

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Sampling dataset
vectorizer_X = TfidfVectorizer(
            max_df=0.95,
            min_df=2,
            max_features=50000,
            stop_words='english'
                              )

#Y
y_all = [
    item[:-1].split(',')#-1 car il y a un ',' à la fin de la ligne
    for item in data_question['SelectedTags']
]

#print(y_train_tag)
lb = tagText.MultiLabelBinarizer()
Y_all = lb.fit_transform(y_all)


# 80/20 split
X_lda_train, X_lda_test, y_lda_train, y_lda_test = train_test_split(
    data_preprocessed, y_all, test_size=0.2,train_size=0.8, random_state=0)
y_lda_train = lb.transform(y_lda_train)
y_lda_test = lb.transform(y_lda_test)

# TF-IDF matrices
X_tfidf_train = vectorizer_X.fit_transform(X_lda_train)
X_tfidf_test = vectorizer_X.transform(X_lda_test)

In [6]:
print(y_lda_test)
print(Y_all)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [7]:
print(len(y_lda_train))

40000


In [8]:
type(X_tfidf_train)
print(X_tfidf_train)

  (0, 36103)	0.08911575469671837
  (0, 2331)	0.09445957924870249
  (0, 8947)	0.187158788758803
  (0, 16691)	0.3429058938039545
  (0, 5432)	0.05647107126682204
  (0, 37204)	0.18638672323487396
  (0, 25407)	0.28964001745015044
  (0, 15978)	0.051339768746363215
  (0, 18302)	0.04811723604358727
  (0, 4078)	0.22568715673826814
  (0, 18124)	0.037693052340521734
  (0, 27069)	0.05911283796019089
  (0, 11237)	0.054134551931285156
  (0, 12750)	0.057130927631255705
  (0, 2624)	0.04466796127998127
  (0, 22488)	0.13886043451045973
  (0, 35798)	0.08059430955539536
  (0, 24577)	0.1454935343755322
  (0, 25405)	0.6228577124065329
  (0, 14279)	0.11336562999779683
  (0, 35523)	0.045601004681814584
  (0, 14205)	0.03924362462107198
  (0, 15584)	0.06373475260148524
  (0, 1543)	0.11545392992819918
  (0, 36440)	0.062106175009756695
  :	:
  (39999, 8595)	0.09497947818377056
  (39999, 28154)	0.06602000083127856
  (39999, 26109)	0.08133630509575526
  (39999, 26490)	0.11663202026137025
  (39999, 25765)	0.08444338

In [9]:
idf = vectorizer_X.idf_
print(idf)

[ 6.89617937  7.37715203  7.50231517 ...  9.19876446 10.49804744
 10.21036537]


In [10]:
dd=dict(zip(vectorizer_X.get_feature_names(), idf))
l=sorted(dd, key=(dd).get)
# print(l)
print(l[0],l[-1])

like firststring


In [11]:
'''
print(dd['like'])
print(dd['consolewritelinesecond'])  # like is most common and forecast is least common among the news headlines.
'''

"\nprint(dd['like'])\nprint(dd['consolewritelinesecond'])  # like is most common and forecast is least common among the news headlines.\n"

In [12]:
print(dd)

{'lowercased': 9.804900263555524, 'nameoptionalofd8': 10.210365371663688, 'nameip': 10.49804744411547, 'leo': 10.210365371663688, 'layerlist': 10.210365371663688, 'nontext': 10.49804744411547, 'metaprogramming': 9.111753082995579, 'findandreplace': 10.49804744411547, 'cacert': 10.210365371663688, 'linestyle': 10.210365371663688, 'spexecutesql': 10.210365371663688, 'httprequestmessage': 9.650749583728265, 'applicationcontextxml': 9.804900263555524, 'wep': 10.49804744411547, 'messageboxshowtest': 10.49804744411547, 'applicationcontext': 8.95760240316832, 'tagname': 8.824071010543797, 'cookiecontainer': 10.210365371663688, 'foosomemethod': 10.49804744411547, 'clones': 8.888609531681368, 'fiddling': 9.111753082995579, 'uiviewcontrollerbasedstatusbarappearance': 10.49804744411547, 'steve': 8.652220753617138, 'floatright': 9.517218191103742, 'arm': 8.95760240316832, 'alerterror': 9.031710375322042, '129': 9.517218191103742, 'comp1': 10.210365371663688, 'declined': 9.517218191103742, 'mykey':

In [13]:
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=42,max_iter=1) 
lda_top=lda_model.fit_transform(X_tfidf_train)


In [14]:
# composition of doc 0 for eg
print("Document 0: ")
for i,topic in enumerate(lda_top[0]):
  print("Topic ",i,": ",topic*100,"%")

Document 0: 
Topic  0 :  81.04959927397279 %
Topic  1 :  1.6752478233821249 %
Topic  2 :  3.576855815234338 %
Topic  3 :  1.6752805138335496 %
Topic  4 :  1.6752742917457388 %
Topic  5 :  1.6753737388337389 %
Topic  6 :  1.6752670076351135 %
Topic  7 :  1.677191475646341 %
Topic  8 :  1.675241414813421 %
Topic  9 :  3.644668644902832 %


In [ ]:
type(lda_top)
print(lda_top)

[[0.81049599 0.01675248 0.03576856 ... 0.01677191 0.01675241 0.03644669]
 [0.05087879 0.77421079 0.02185689 ... 0.02185689 0.02185689 0.02191177]
 [0.01403764 0.04584738 0.0140266  ... 0.05010771 0.0140266  0.80584765]
 ...
 [0.13368694 0.0146401  0.01463226 ... 0.01463229 0.01463226 0.74922247]
 [0.24183268 0.01913787 0.05126342 ... 0.01913539 0.01913539 0.57295296]
 [0.64406955 0.08522699 0.01613776 ... 0.01613776 0.01613776 0.15773911]]


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(
        n_estimators=100,
        max_depth=2,
        random_state=0
)
#lda_top=lda_model.fit_transform(X_tfidf_train)
clf.fit(lda_top, y_lda_train)

In [ ]:
classes = lb.classes_

no_top_words = 5
text = "git is the best source code"
text = tagText.preprocessing(text)
text = [text]
mytext = vectorizer_X.transform(text)
text_projection = lda_model.transform(mytext)
'''
print (text_projection)
print (text_projection.shape)
'''
predicted = clf.predict_proba(text_projection)
'''
print (predicted)
print (len(predicted))
'''
tempTag = [(1-item[0][0]) for item in predicted]
list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
'''
tempTag = predicted[0]
list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
'''
'''
print (predicted)
print (predicted.shape)
'''
list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
#print (list_id_sorted)

list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
#print(list_id_sorted_suggested)

print (str([classes[id] for id in list_id_sorted_suggested]))

In [ ]:
def scoring(x_test, y_true, clf, lb, mode_supervise_with_lda = False, lda_model = None):
    nb_tag_1 = 0.0
    nb_tag_5 = 0.0
    classes = lb.classes_
    print(x_test.shape[0])
    no_top_words = 5
    for i in range(x_test.shape[0]):
        #for i in range(x_test.shape[0]):
        text_projection = x_test
        if(mode_supervise_with_lda):
            text_projection = lda_model.transform(x_test[i])
        '''
        print (text_projection)
        print (text_projection.shape)
        '''
        predicted = clf.predict_proba(text_projection)
        '''
        print (predicted)
        print (len(predicted))
        '''
        tempTag = [(1-item[0][0]) for item in predicted]
        list_id = [[i, x] for i, x in enumerate(tempTag) if x > 0.0050]
        '''
        tempTag = predicted[0]
        list_id = [[i, x] for i, x in enumerate(tempTag) ]#if x > 0.0050]
        '''
        '''
        print (predicted)
        print (predicted.shape)
        '''
        list_id_sorted = sorted(list_id, reverse=True, key=lambda x: x[1])
        #print (list_id_sorted)

        list_id_sorted_suggested = [x[0] for i, x in enumerate(list_id_sorted[:-no_top_words - 1:-1])]
        #print(list_id_sorted_suggested)
        prediction = [classes[id] for id in list_id_sorted_suggested]
        #print (str(prediction))
        
        
        l_y = [[i, x] for i, x in enumerate(y_true[i]) if x > 0]
        l_y_tagged = [x[0] for i, x in enumerate(l_y[:-no_top_words - 1:-1])]
        l_y_tags = [classes[id] for id in l_y_tagged]
        #print (l_y_tags)
        
        check_1 = False
        check_1 = any(item in prediction for item in l_y_tags)

        if check_1 is True:
            nb_tag_1 = nb_tag_1 + 1
        """    print("The list {} contains some elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """

        check_5 = False
        check_5 = all(item in prediction for item in l_y_tags)
        if check_5 is True:
            nb_tag_5 = nb_tag_5 + 1
        """    print("The list {} contains all elements of the list {}".format(prediction, l_y_tags))    
        else :
            print("No, List1 doesn't have any elements of the List2.")
        """
        #str([tag for tag in y_true[i]]if tag ==1)
    return nb_tag_1, (100.0 * nb_tag_1 / float(x_test.shape[0])), nb_tag_5, (100.0 * nb_tag_5 / float(x_test.shape[0]))

scoring(X_tfidf_test, y_lda_test, clf, lb, True, lda_model)

<p style="font-family: Arial; font-size:1.75em;color:#2462C0; font-style:bold"><br>
IV.2-a - Data - Tag generator - Unsupervised - LDA</p>

In [ ]:
def lda_prepare_tag(tagText, data_preprocessed, data_tag, no_tropics=32):
        '''
        prepare lda, topic ad tf vectorizer from data preprocessed
        '''
        from sklearn.ensemble import RandomForestClassifier
        
        tagText.n_topic = no_tropics
        documents = data_preprocessed[0:tagText.precision].unique()
        lda_tf, lda_tf_vectorizer = tagText.lda_init(documents)
        
        lda, score, perplexity = tagText.lda_train(lda_tf, no_tropics)
        
        print("Log Likelihood: ", score)
        print("Perplexity: ", perplexity)

        # See model parameters
        print(lda.get_params())

        lda_topicnames = ["Topic" + str(i) for i in range(lda.n_topics)]
        # Topic-Keyword Matrix
        lda_df_topic_keywords = tagText.pd.DataFrame(lda.components_)
        
        # Assign Column and Index
        lda_df_topic_keywords.columns = lda_tf_vectorizer.get_feature_names()
        lda_df_topic_keywords.index = lda_topicnames
        
        #tag
        topic_word = lda.topic_word_
        print("type(topic_word): {}".format(type(topic_word)))
        print("shape: {}".format(topic_word.shape))
        
        clf = RandomForestClassifier(
                n_estimators=100,
                max_depth=2,
                random_state=0
        )
        clf.fit(lda, y_lda_train)
        
        return lda, lda_df_topic_keywords, lda_tf_vectorizer, clf

In [ ]:
lda_tagText, df_topic_keywords, tf_vectorizer, clf = lda_prepare_tag(tagText, data_preprocessed, X_tfidf_train, y_lda_train, 32)


In [ ]:
def average_jaccard(y_true,y_pred):

    ''' It calculates Jaccard similarity coefficient score for each instance,and
    it finds their averange in percentage
    Parameters:
    y_true: truth labels
    y_pred: predicted labels
    '''
    jacard = np.minimum(y_true,y_pred).sum(axis=1) / np.maximum(y_true,y_pred).sum(axis=1)
    
    return jacard.mean()*100

In [ ]:
# most important words for each topic
vocab = vectorizer_X.get_feature_names()

for i, comp in enumerate(lda_tagText.components_):
    vocab_comp = zip(vocab, comp)
    sorted_words = sorted(vocab_comp, key= lambda x:x[1], reverse=True)[:10]
    print("Topic "+str(i)+": ")
    for t in sorted_words:
        print(t[0],end=" ")
    print("\n")